In [1]:
import pandas as pd
import numpy as np
import re
from kiwipiepy import Kiwi
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import pymysql


In [2]:
con = pymysql.connect(
    host='localhost',
    user='root',
    password='1111',
    db='yorehalre',
    charset='utf8'
)

data = pd.read_sql("SELECT recipeID, recipe_name, summary, ingredient_name, category FROM recipe_info", con)
data_re = pd.read_sql("SELECT category, recipeID, recipe_name, user_id, rating, review_content FROM user_review", con)

con.close()

C:\Users\SSO\AppData\Local\Temp\ipykernel_20776\2245140572.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("SELECT recipeID, recipe_name, summary, ingredient_name, category FROM recipe_info", con)
C:\Users\SSO\AppData\Local\Temp\ipykernel_20776\2245140572.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_re = pd.read_sql("SELECT category, recipeID, recipe_name, user_id, rating, review_content FROM user_review", con)


In [3]:
# data = pd.read_csv('../data/weaviate_data(임시).csv')
# data_re = pd.read_csv('../data/user_review.csv')

In [4]:
source = []
for ingredient_name in data['ingredient_name']:
    ingredient_name_str = str(ingredient_name)
    if '양념' in ingredient_name_str:
        ingredient_matches = re.findall(r'\[.*?\](.*?)\양념', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    else:
        ingredient_matches = re.findall(r'\[.*?\](.*)', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    # if source_per_ingredient:
    source.append(source_per_ingredient)

In [5]:
len(source)

874

In [6]:
sum(1 for item in source if not item)

32

In [7]:
data['main_source'] = source

In [8]:
data_re.head(3)

,category,recipeID,recipe_name,user_id,rating,review_content
0,간 건강,765,카레 해물 파스타,3366,3,참고해서 잘 해먹었습니다~ 우유가 없어서 카레를 그냥 재료들과 볶았는데 나쁘지 않았어요
1,간 건강,768,생생정보 오이미역냉국 황금레시피,1013,5,애들이 맛있다고 잘먹어요..
2,간 건강,768,생생정보 오이미역냉국 황금레시피,1173,5,정말 맛나요~


In [9]:
df = pd.merge(data, data_re, on=['recipeID', 'recipe_name', 'category'], how='outer')

In [10]:
kiwi = Kiwi()
def extract_keywords(text):
    stop_words = ['물', '약간', '적당', '적당량', '조금', '때', '레시피', '요리', '저', '거', '것', '수', '나']
    keywords = []
    text = text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    for title in text:
        titles = []
        result = kiwi.analyze(title)
        for token, tag, _ , _ in result[0][0]:
            if tag[0] in 'NN':
                if token not in stop_words and not re.match(r'.*\W.*', token):
                    titles.append(token.strip())
        keywords.append(titles)
    return keywords

In [11]:
name = extract_keywords(df['recipe_name'].apply(str))
summary = extract_keywords(df['summary'].apply(str))
source = extract_keywords(df['main_source'].apply(str))

In [12]:
review = extract_keywords(df['review_content'].apply(str))


In [13]:
list_all = []

for item_1, item_2, item_3, item_4 in zip(name, summary, df['main_source'], review):
    combined_list = item_1 + item_2 + item_3 + item_4
    list_all.append(combined_list)

In [14]:
df['all'] = list_all

In [15]:
all_keyword = []
for i in df['all']:
    for item in i:
        all_keyword.append(item)
print(len(all_keyword))

86131


In [16]:
unique_keyword = set(all_keyword)
# print(unique_keyword)
print(len(unique_keyword))


4022


In [17]:
'우엉' in unique_keyword

True

In [18]:
total_recipe_count = len(set(data['recipe_name']))
keyword_count_dict = dict.fromkeys(unique_keyword)

for keyword_list in df['all']:
    for keyword in keyword_list:
        if keyword_count_dict[keyword.strip()] == None:
            keyword_count_dict[keyword.strip()] = 1
        else:
            keyword_count_dict[keyword.strip()] += 1

In [19]:
keyword_idf = dict()
for each_keyword in keyword_count_dict:
    keyword_idf[each_keyword] = np.log10(total_recipe_count/keyword_count_dict[each_keyword])

In [20]:
sorted_data_dict = dict(sorted(keyword_idf.items(), key=lambda item: item[1], reverse=True))

In [21]:
all_category = data['category'].explode().tolist()
all_keyword = df['all'].explode().tolist()
unique_keyword = set(all_keyword)
unique_category = set(all_category)

In [22]:
unique_category

{'간 건강',
 '골다공증',
 '노화방지',
 '머슬업 다이어트',
 '변비',
 '스트레스 해소',
 '암 예방',
 '위 건강',
 '장 건강',
 '저칼로리 다이어트',
 '치매예방',
 '피로회복',
 '혈당조절',
 '혈액순환',
 '호흡기 건강'}

## category 별 데이터프레임

In [23]:
category_representation = pd.DataFrame(columns=sorted(unique_category), index=df['recipeID'])
print(category_representation.head(1))

         간 건강 골다공증 노화방지 머슬업 다이어트   변비 스트레스 해소 암 예방 위 건강 장 건강 저칼로리 다이어트 치매예방  \
recipeID                                                                      
0         NaN  NaN  NaN      NaN  NaN     NaN  NaN  NaN  NaN       NaN  NaN   

         피로회복 혈당조절 혈액순환 호흡기 건강  
recipeID                        
0         NaN  NaN  NaN    NaN  


In [24]:
for category in df['category']:
    # 해당하는 카테고리에는 1을 넣고 아니면 0을 넣음
    category_representation[category] = np.where(df['category'] == category, 2, None)

In [25]:
print(category_representation.tail(5))

          간 건강  골다공증  노화방지 머슬업 다이어트    변비 스트레스 해소  암 예방  위 건강  장 건강 저칼로리 다이어트  \
recipeID                                                                        
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
873       None  None  None     None  None    None  None  None  None      None   

         치매예방  피로회복  혈당조절  혈액순환 호흡기 건강  
recipeID                                
872         2  None  None  None   None  
872         2  None  None  None   None  
872         2  None  None  None   None  
872         2  None  None  None   None  
873         2  None  None  None   None  


## keyword 별 데이터프레임

In [26]:
keyword_representation = pd.DataFrame(columns=sorted(unique_keyword), index=df['recipeID'])
print(keyword_representation.head(1))

          2손다  3스픈    ᆢ  ᆢ저는    ㅋ   ㅋ넘   ㅋ존    ㅎ   ㅡ넘    가  ...  흰우유   흰자  \
recipeID                                                    ...             
0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

           흰콩  흰후추 흰후추가루    히   히트   힌트   힐링    힘  
recipeID                                           
0         NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  

[1 rows x 4022 columns]


In [27]:
for index in tqdm(range(len(df['recipeID']))):
    keywords = df['all'][index]
    recipeID = df['recipeID'].iloc[index]
    for keyword in keywords:
        keyword_idf_value = keyword_idf.get(keyword.strip(), None) 
        if keyword_idf_value is not None:
            keyword_representation.at[recipeID, keyword] = keyword_idf_value

# keyword_representation.sort_index(inplace=True)

  0%|          | 0/4025 [00:00<?, ?it/s]

100%|██████████| 4025/4025 [00:15<00:00, 263.58it/s]


In [28]:
keyword_representation.head(5)

,2손다,3스픈,ᆢ,ᆢ저는,ㅋ,ㅋ넘,ㅋ존,ㅎ,ㅡ넘,가,...,흰우유,흰자,흰콩,흰후추,흰후추가루,히,히트,힌트,힐링,힘
recipeID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 카테고리 + 키워드

In [29]:
recipe_representation = pd.concat([category_representation, keyword_representation], axis=1).fillna(0)


In [30]:
recipe_representation = recipe_representation.drop_duplicates()
print(recipe_representation.shape)
print(recipe_representation.describe())

(871, 4037)
             간 건강        골다공증        노화방지    머슬업 다이어트          변비     스트레스 해소  \
count  871.000000  871.000000  871.000000  871.000000  871.000000  871.000000   
mean     0.137773    0.137773    0.133180    0.137773    0.130884    0.137773   
std      0.506813    0.506813    0.498908    0.506813    0.494893    0.506813   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
75%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
max      2.000000    2.000000    2.000000    2.000000    2.000000    2.000000   

             암 예방        위 건강        장 건강   저칼로리 다이어트  ...         흰우유  \
count  871.000000  871.000000  871.000000  871.000000  ...  871.000000   
mean     0.126292    0.137773    0.140069    0.137773  ...    0.005980   
std      0.486730    0.506813    0.5

In [31]:
def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])
    
    return result_df

In [32]:
cs_df = cos_sim_matrix(recipe_representation, recipe_representation)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,861,862,863,864,865,866,867,868,869,870
recipeID,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.104553,0.099753,0.123909,0.115700,0.168121,0.093792,0.096656,0.212477,0.085763,...,0.000000,0.000173,0.000000,0.000000,0.000000,0.000106,0.000000,0.023696,0.014620,0.000000
1,0.104553,1.000000,0.081507,0.104183,0.097282,0.143414,0.079756,0.082452,0.181252,0.083657,...,0.000000,0.029175,0.000000,0.000000,0.000000,0.001682,0.002344,0.000000,0.001828,0.000000
2,0.099753,0.081507,1.000000,0.117808,0.204224,0.131063,0.072887,0.075367,0.165642,0.066859,...,0.000000,0.000009,0.000000,0.009905,0.000000,0.010524,0.000000,0.000000,0.000000,0.000050
3,0.123909,0.104183,0.117808,1.000000,0.115291,0.162801,0.090537,0.093617,0.205753,0.085460,...,0.000000,0.001542,0.000000,0.000037,0.000000,0.000938,0.002778,0.000000,0.002166,0.000030
4,0.115700,0.097282,0.204224,0.115291,1.000000,0.152015,0.084539,0.100553,0.194003,0.079900,...,0.000786,0.001976,0.012838,0.000000,0.000281,0.020494,0.038640,0.000000,0.002704,0.000029


In [33]:
cs_df.shape

(871, 871)

In [34]:
print(cs_df[0].sort_values(ascending=False)[1:4])

recipeID
8           0.212477
28          0.204769
35          0.203645
Name: 0, dtype: float64


In [35]:
print(data.loc[0])
print(data.loc[8])
print(data.loc[28])
print(data.loc[35])

recipeID                                                           0
recipe_name                                                 우엉양배추볶음밥
summary                                       찬밥이 기다려지는 우엉양배추볶음밥입니다.
ingredient_name    [재료] 찬 밥  적당량 양배추  적당량 우엉  적당량 느타리버섯  조금 다시마  ...
category                                                   저칼로리 다이어트
main_source        [찬, 밥, 적당량, 양배추, 적당량, 우엉, 적당량, 느타리버섯, 조금, 다시마,...
Name: 0, dtype: object
recipeID                                                           8
recipe_name                                                    곤약 무침
summary                                    다이어트에 좋다는 곤약무침을 만들어 보았네요.
ingredient_name    [재료] 곤약(250g)  1개 돌나물  100g 당근  10g 양파  10g [양...
category                                                   저칼로리 다이어트
main_source                                        [곤약, 돌나물, 당근, 양파]
Name: 8, dtype: object
recipeID                                                          28
recipe_name                                              

In [36]:
train_df, test_df = train_test_split(data_re, test_size=0.2, random_state=1234)

In [37]:
test_userids = list(set(test_df.user_id.values))

In [38]:
result_df = pd.DataFrame()

for user_id in tqdm(test_userids):
    user_record_df = train_df.loc[train_df.user_id == int(user_id), :]
    
    user_sim_df = cs_df.loc[user_record_df['recipeID']]
    user_rating_df = user_record_df[['rating']]

    sim_sum = np.sum(user_sim_df.T.to_numpy(), -1)     # 각 열의 값을 합산
    prediction = np.matmul(user_sim_df.T.to_numpy(), user_rating_df.to_numpy()).flatten() / (sim_sum+1)
    
    # 프린트 하는 과정
    prediction_df = pd.DataFrame(prediction, index=cs_df.index).reset_index()
    prediction_df.columns = ['recipeID', 'pred_rating']
    prediction_df = prediction_df[['recipeID', 'pred_rating']][prediction_df.recipeID.isin(test_df[test_df.user_id == user_id]['recipeID'])]
    
    temp_df = prediction_df.merge(test_df[test_df.user_id == user_id], on='recipeID')
    result_df = pd.concat([result_df, temp_df], axis=0)

100%|██████████| 695/695 [00:08<00:00, 84.89it/s] 


In [39]:
result_df.head(2)

,recipeID,pred_rating,category,recipe_name,user_id,rating,review_content
0,623,0.0,혈당조절,구운애호박 무침,2660,5,애호박이 딱 이만큼 남아서 어떡할까 고민하다가 따라해 보았어요~ 맛있고 요리하기도 ...
0,479,0.0,혈액순환,부추무침,3269,5,새콤 달콤 맛있어요 탐나는 레시피에요^^


In [40]:
print(f"RMSE: {sqrt(mean_squared_error(result_df['rating'].values, result_df['pred_rating'].values))}")

RMSE: 4.869690033239285


# 협업필터링

## 아이템 기반

In [41]:
train_df, test_df = train_test_split(data_re, test_size=0.2, random_state=1234)

In [42]:
train_df.shape, test_df.shape

((2824, 6), (706, 6))

In [43]:
user_ids = sorted(set(train_df['user_id'].values))
recipe_ids = sorted(set(train_df['recipeID'].values))

print(f'유저 수: {len(user_ids)}, 레시피 수: {len(recipe_ids)}')


유저 수: 2645, 레시피 수: 351


In [44]:
sparse_matrix = pd.DataFrame(index=recipe_ids, columns=user_ids)
sparse_matrix

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
train_df.head(10)

,category,recipeID,recipe_name,user_id,rating,review_content
1348,스트레스 해소,376,불닭볶음면+틈새라면=지옥의 염라면 만들기,765,5,맛있네요!
3259,혈액순환,479,부추무침,1108,5,정말 맛있어요^^ 잘 이용하겠습니당
235,골다공증,197,통조림으로 만든 집밥 백선생 일본식 꽁치조림,62,5,매번 레시피 잊어버리는데 만 개의 레시피를 보고 다시 떠올리곤 합니다. ㅎ
671,머슬업 다이어트,84,고등어무조림,2087,5,레시피대로햇더니 맛나네요~~생강가루 조금 첨가했어요 울아찌 맛있다고 한그릇 뚝딱햇어요
89,간 건강,784,비오는날에는 ★새우부추전,2569,5,간단하게 할수 있고 영양 좋은 음식이라\r\n좋았읍니다.
2655,치매예방,869,절대 후회없는 깻잎김치 황금레시피^^,2083,5,색깔이며 맛이며 좋아요^^\r\n다만 깻잎두장씩 해서했는데도 양념이 계량된데로하니 ...
507,머슬업 다이어트,84,고등어무조림,1107,5,이대로만 하시면 엄마가 해주시던 고등어조림 맛이 날겁니다. \r\n저는 무가 푹 익...
29,간 건강,768,생생정보 오이미역냉국 황금레시피,733,5,미역오이냉국이예여\r\n함올려봣어여\r\n맛잇겟죠
218,골다공증,196,송일국 꽁치쌈장 만들기 ★,3219,5,강추! 너무 맛있어요. 상추쌈에도 굿~ 입니다. 감사해요.
1130,머슬업 다이어트,110,닭가슴살 갈비구이,3169,5,만들기 쉽고 맛있어요~ 저는 간장 4스푼 넣었구요. 닭만 먹으면 조금 짠데 밥이나 ...


In [46]:
train_df_grouped = train_df.groupby(['recipeID', 'user_id'])['rating'].mean().reset_index()
sparse_matrix = train_df_grouped.pivot(index='recipeID', columns='user_id', values='rating')
sparse_matrix.index.name = 'recipeID'
sparse_matrix

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
recipeID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index=a.index)

    return cossim_df

In [48]:
item_sparse_matrix = sparse_matrix.fillna(0)
item_sparse_matrix

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
recipeID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
item_max_values = sparse_matrix.max(axis=1)
item_min_values = sparse_matrix.max(axis=1)
item_mean_values = sparse_matrix.mean(axis=1)
item_count_values = sparse_matrix.count(axis=1)
user_max_values = sparse_matrix.max(axis=0)
user_min_values = sparse_matrix.max(axis=0)
user_mean_values = sparse_matrix.mean(axis=0)
user_count_values = sparse_matrix.count(axis=0)

In [50]:
print(item_max_values, item_min_values, item_mean_values, item_count_values)

recipeID
1      5.0
2      5.0
4      5.0
8      5.0
9      5.0
      ... 
865    5.0
868    5.0
869    5.0
870    5.0
872    5.0
Length: 351, dtype: float64 recipeID
1      5.0
2      5.0
4      5.0
8      5.0
9      5.0
      ... 
865    5.0
868    5.0
869    5.0
870    5.0
872    5.0
Length: 351, dtype: float64 recipeID
1      3.666667
2      5.000000
4      5.000000
8      5.000000
9      4.875000
         ...   
865    4.882353
868    5.000000
869    4.831325
870    4.800000
872    5.000000
Length: 351, dtype: float64 recipeID
1       3
2       1
4       1
8       2
9       8
       ..
865    17
868     2
869    83
870     5
872     6
Length: 351, dtype: int64


In [51]:
for i, row in enumerate(item_sparse_matrix.values):
    # 0이 아닌 데이터가 있는 칸의 수 계산
    nonzero_count = np.count_nonzero(row)
    
    # 행의 평균값 계산
    row_mean = np.mean(row[row != 0])
    
    # 데이터가 있는 칸의 수의 2배에 해당하는 위치에 평균값 넣기
    for j in range(nonzero_count * 5):
        item_sparse_matrix.iloc[i, j] = row_mean

In [52]:
item_cossim_df = cossim_matrix(item_sparse_matrix, item_sparse_matrix)
item_cossim_df

,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
recipeID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.470860,0.470860,0.665897,0.499231,0.470860,0.812355,0.470860,0.662455,0.470860,...,0.706290,0.665897,0.470860,0.470860,0.533905,0.342477,0.665897,0.156624,0.631360,0.584864
2,0.470860,1.000000,0.833333,0.589256,0.294515,0.833333,0.479238,0.833333,0.586210,0.833333,...,0.416667,0.589256,0.833333,0.833333,0.314970,0.202040,0.589256,0.092398,0.372463,0.345033
4,0.470860,0.833333,1.000000,0.589256,0.294515,0.833333,0.479238,0.833333,0.586210,0.833333,...,0.416667,0.589256,0.833333,0.833333,0.314970,0.202040,0.589256,0.092398,0.372463,0.345033
8,0.665897,0.589256,0.589256,1.000000,0.416507,0.589256,0.677746,0.589256,0.829027,0.589256,...,0.589256,0.833333,0.589256,0.589256,0.445435,0.285727,0.833333,0.143738,0.526742,0.487950
9,0.499231,0.294515,0.294515,0.416507,1.000000,0.294515,0.508114,0.294515,0.414354,0.294515,...,0.589030,0.416507,0.294515,0.294515,0.779213,0.571236,0.416507,0.267940,0.658175,0.731645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.342477,0.202040,0.202040,0.285727,0.571236,0.202040,0.348571,0.202040,0.284251,0.202040,...,0.404079,0.285727,0.202040,0.202040,0.534547,1.000000,0.285727,0.384629,0.451513,0.501914
868,0.665897,0.589256,0.589256,0.833333,0.416507,0.589256,0.677746,0.589256,0.829027,0.589256,...,0.589256,0.833333,0.589256,0.589256,0.445435,0.285727,1.000000,0.130671,0.526742,0.487950
869,0.156624,0.092398,0.092398,0.143738,0.267940,0.092398,0.183936,0.092398,0.146245,0.092398,...,0.194037,0.130671,0.092398,0.092398,0.258676,0.384629,0.130671,1.000000,0.215394,0.237191


In [53]:
userId_grouped = train_df.groupby('user_id')
# userId_grouped.indices.keys() : user_id
# item_sparse_matrix.index : recipe_id
item_prediction_result_df = pd.DataFrame(index=list(userId_grouped.indices.keys()), columns=item_sparse_matrix.index)
item_prediction_result_df

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
for user_id, group in tqdm(userId_grouped):
    user_sim = item_cossim_df.loc[group['recipeID']]   # 유저의 레시피 유사도
    user_rating = group['rating']   # 유저의 평점

    # 사용자가 평가한 레시피들의 유사도 총합을 계산
    sim_sum = user_sim.sum(axis=0)
    
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    item_prediction_result_df.loc[user_id] = pred_ratings

  1%|          | 30/2645 [00:00<00:09, 289.23it/s]

100%|██████████| 2645/2645 [00:05<00:00, 483.20it/s]


In [55]:
item_prediction_result_df.head(10)

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
0,2.051028,1.904901,1.904901,2.326758,1.515746,1.904901,2.072397,1.904901,2.320314,1.904901,...,1.904901,2.326758,1.904901,1.904901,1.587595,1.235721,2.326758,0.652664,1.774534,1.687968
1,2.281423,1.655683,1.655683,2.059066,1.721105,1.655683,2.303318,1.655683,2.052793,1.655683,...,2.130742,2.059066,1.655683,1.655683,1.797665,1.323768,2.059066,0.70697,1.99488,1.90391
10,1.519286,1.023856,1.023856,1.33474,2.106549,1.023856,1.538005,1.023856,1.329677,1.023856,...,1.699669,1.33474,1.023856,1.023856,2.026076,2.035689,1.33474,1.222733,1.826312,1.950644
100,2.281423,1.655683,1.655683,2.059066,1.721105,1.655683,2.303318,1.655683,2.052793,1.655683,...,2.130742,2.059066,1.655683,1.655683,1.797665,1.387529,2.059066,0.747059,1.99488,1.90391
1000,0.592646,0.367485,0.367485,0.549243,0.953997,0.367485,0.681896,0.367485,0.590892,0.434594,...,0.713995,0.504348,0.434594,0.367485,0.887742,1.263041,0.504348,2.128059,0.752918,0.868471
1001,0.716145,0.448842,0.448842,0.665063,1.111935,0.448842,0.820531,0.448842,0.675096,0.448842,...,0.857859,0.612003,0.448842,0.448842,1.034667,1.457815,0.612003,2.219523,0.928388,0.983933
1002,0.62914,0.391346,0.391346,0.583419,0.987935,0.391346,0.722985,0.391346,0.592389,0.391346,...,0.756679,0.536067,0.462377,0.391346,0.938539,1.328375,0.536067,2.20555,0.797496,0.894847
1003,1.043072,0.672911,0.672911,0.901391,1.527015,0.672911,1.057706,0.672911,0.897569,0.672911,...,1.186183,0.901391,0.672911,0.672911,1.457529,1.967883,0.901391,1.682669,1.289515,1.393347
1004,0.710816,0.445295,0.445295,0.660055,1.10442,0.445295,0.814573,0.445295,0.670025,0.445295,...,0.851685,0.607338,0.445295,0.445295,1.027536,1.43609,0.607338,2.225564,0.896544,0.977066
1005,0.677074,0.422915,0.422915,0.62837,1.056597,0.422915,0.776798,0.422915,0.637932,0.422915,...,0.812524,0.577847,0.422915,0.422915,1.027573,1.388924,0.577847,2.5,0.886107,0.958586


In [56]:
print(item_prediction_result_df.shape)

(2645, 351)


## 유저 기반

In [57]:
user_sparse_matrix = sparse_matrix.fillna(0).transpose()
user_sparse_matrix.head(5)

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
user_sparse_matrix.shape

(2645, 351)

In [59]:
for i, row in enumerate(user_sparse_matrix.values):
    nonzero_count = np.count_nonzero(row)
    
    row_mean = np.mean(row[row != 0])
    for j in range(nonzero_count * 5):
        user_sparse_matrix.iloc[i, j] = row_mean

In [60]:
user_sparse_matrix

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
990,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
991,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)
user_cossim_df

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.481125,0.833333,0.833333,0.833333
1,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.481125,0.833333,0.833333,0.833333
10,0.833333,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.481125,0.833333,0.833333,0.833333
100,0.833333,0.833333,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.481125,0.833333,0.833333,0.833333
1000,0.833333,0.833333,0.833333,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.481125,0.833333,1.000000,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.833333,1.000000,0.833333,1.000000,0.481125,0.833333,0.833333,1.000000
990,0.481125,0.481125,0.481125,0.481125,0.481125,0.481125,0.481125,0.481125,0.481125,0.481125,...,0.481125,0.481125,0.481125,0.481125,0.481125,0.481125,1.000000,0.481125,0.481125,0.481125
991,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.481125,1.000000,0.833333,0.833333


In [62]:
recipeId_grouped = train_df.groupby('recipeID')
user_prediction_result_df = pd.DataFrame(index=list(recipeId_grouped.indices.keys()), columns=user_sparse_matrix.index)
user_prediction_result_df

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
for recipeID, group in tqdm(recipeId_grouped):
    user_sim = user_cossim_df.loc[group['user_id']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)
    
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    user_prediction_result_df.loc[recipeID] = pred_ratings

  4%|▍         | 15/351 [00:00<00:02, 142.54it/s]

100%|██████████| 351/351 [00:01<00:00, 176.23it/s]


In [64]:
user_prediction_result_df = user_prediction_result_df.transpose()

In [65]:
user_prediction_result_df

,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
user_id,,,,,,,,,,,,,,,,,,,,,
0,2.685911,2.386128,2.386128,3.022409,4.298555,1.916272,3.101821,2.272727,2.452122,2.272727,...,3.846154,3.125,2.272727,2.272727,4.241189,4.512784,3.125,4.756572,3.708035,4.166667
1,2.685911,2.386128,2.386128,3.022409,4.298555,1.916272,3.101821,2.272727,2.452122,2.272727,...,3.846154,3.125,2.272727,2.272727,4.241189,4.512784,3.125,4.756572,3.708035,4.166667
10,2.685911,2.386128,2.386128,3.022409,4.298555,1.916272,3.101821,2.272727,2.452122,2.272727,...,3.846154,3.125,2.272727,2.272727,4.241189,4.512784,3.125,4.756955,3.708035,4.166667
100,2.685911,2.386128,2.386128,3.022409,4.298555,1.916272,3.101821,2.272727,2.452122,2.272727,...,3.846154,3.125,2.272727,2.272727,4.241189,4.5149,3.125,4.756165,3.708035,4.166667
1000,2.685911,2.386128,2.386128,3.022409,4.298555,1.916272,3.101821,2.272727,2.483946,2.272727,...,3.846154,3.125,2.272727,2.272727,4.241189,4.512784,3.125,4.757276,3.708035,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,2.685911,2.386128,2.386128,3.022409,4.298555,1.916272,3.101821,2.272727,2.452122,2.272727,...,3.846154,3.125,2.272727,2.272727,4.241189,4.514776,3.125,4.757177,3.736149,4.166667
990,2.246105,1.725705,1.725705,2.765578,3.942467,2.088872,2.758167,1.830127,2.187278,1.830127,...,3.290307,2.451905,1.624188,1.624188,3.905229,4.414691,2.451905,4.730962,3.651305,3.713579
991,2.685911,2.386128,2.386128,3.022409,4.298555,1.916272,3.101821,2.272727,2.452122,2.272727,...,3.846154,3.125,2.272727,2.272727,4.241189,4.512784,3.125,4.756572,3.708035,4.166667


In [66]:
print(user_prediction_result_df.shape)
print(item_prediction_result_df.shape)

(2645, 351)
(2645, 351)


In [67]:
test_df.head()

,category,recipeID,recipe_name,user_id,rating,review_content
2766,피로회복,441,문어 꽈리고추 조림,1421,3,문어가 없어서 오징어로 대체해봤어요
63,간 건강,779,새우 브로콜리 볶음,1296,5,새우가 냉동실에 많아 다 쏟아붓고했더니 ㅋㅋ 많은양에 양념을 못맞춰 이것더것 조금씩...
3525,호흡기 건강,586,겨울철 가족의 건강을 지켜주는 대추고(차) 만드는법,3177,5,진짜 열심히 만들었는데 만들고 보니 된장 같이 생겼네요 ㅋㅋ 물에 타먹으니 맛있어요 ^^
2126,장 건강,325,느타리버섯양념구이,596,5,맛있어요~ 벌써 여러 번 해 먹었답니다 ^^
25,간 건강,768,생생정보 오이미역냉국 황금레시피,52,5,너무맞있게먹었요.


In [68]:
# def evaluate(test_df, prediction_result_df):
#   groups_with_movie_ids = test_df.groupby(by='recipeID')
#   groups_with_user_ids = test_df.groupby(by='user_id')
  
#   # 트레인 테스트 df에도 있는 무비 ID와 유저 아이디 가지고 오기
#   intersection_movie_ids = sorted(list(set(list(prediction_result_df.columns)). \
#       intersection(set(list(groups_with_movie_ids.indices.keys())))))
#   intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

#   print(len(intersection_movie_ids))
#   print(len(intersection_user_ids))
  
#   compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_movie_ids]
  
#   grouped = test_df.groupby(by='user_id')
# #   result_df = pd.DataFrame(columns=['rmse'])
  
#   for user_id, group in tqdm(grouped):
#       if user_id in intersection_user_ids:
#           pred_ratings = compressed_prediction_df.loc[user_id][compressed_prediction_df.loc[user_id].index.\
#               intersection(list(group['recipeID'].values))]
#           pred_ratings = pred_ratings.to_frame(name='rating').reset_index().\
#               rename(columns={'index':'recipeID','rating':'pred_rating'})
#           actual_ratings = group[['rating', 'recipeID']].rename(columns={'rating':'actual_rating'})
          
#           final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['recipeID'])
#           final_df = final_df.round(4)
          
#   return final_df
  

In [69]:
def evaluate(test_df, prediction_result_df):
    groups_with_recipe_ids = test_df.groupby(by='recipeID')
    groups_with_user_ids = test_df.groupby(by='user_id')
  
    intersection_recipe_ids = sorted(list(set(list(prediction_result_df.columns)).intersection(set(list(groups_with_recipe_ids.indices.keys())))))
    intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

    print(len(intersection_recipe_ids))
    print(len(intersection_user_ids))
  
    compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_recipe_ids]
  
    grouped = test_df.groupby(by='user_id')
    result_df_list = []  # List to store final_df for each user
  
    for user_id, group in tqdm(grouped):
        if user_id in intersection_user_ids:
            pred_ratings = compressed_prediction_df.loc[user_id][compressed_prediction_df.loc[user_id].index.\
                intersection(list(group['recipeID'].values))]
            pred_ratings = pred_ratings.to_frame(name='rating').reset_index().\
                rename(columns={'index':'recipeID','rating':'pred_rating'})
            actual_ratings = group[['rating', 'recipeID']].rename(columns={'rating':'actual_rating'})
          
            final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['recipeID'])
            final_df = final_df.round(4)
            
            result_df_list.append(final_df)  # Append final_df to result_df_list

    # Concatenate all dataframes in result_df_list
    result_df = pd.concat(result_df_list, ignore_index=True)
    
    return result_df

In [70]:
evaluate(test_df, user_prediction_result_df)

171
69


100%|██████████| 695/695 [00:00<00:00, 972.26it/s] 


,actual_rating,recipeID,pred_rating
0,5,73,4.48088
1,5,84,4.984869
2,5,613,4.177674
3,4,750,2.272727
4,5,618,4.858417
...,...,...,...
68,5,356,4.711397
69,5,743,4.666379
70,5,845,4.719062
71,5,625,4.480456


In [71]:
evaluate(test_df, item_prediction_result_df)

171
69


 36%|███▋      | 252/695 [00:00<00:00, 1221.15it/s]

100%|██████████| 695/695 [00:00<00:00, 934.73it/s] 


,actual_rating,recipeID,pred_rating
0,5,73,1.726988
1,5,84,1.97186
2,5,613,2.910622
3,4,750,1.483098
4,5,618,0.707263
...,...,...,...
68,5,356,1.246977
69,5,743,2.237751
70,5,845,2.181232
71,5,625,2.747818


In [72]:
result_df = evaluate(test_df, user_prediction_result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

171
69


  8%|▊         | 53/695 [00:00<00:01, 515.72it/s]

100%|██████████| 695/695 [00:00<00:00, 1400.45it/s]


RMSE: 0.9984469953214047


In [73]:
result_df = evaluate(test_df, item_prediction_result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

171
69


  0%|          | 0/695 [00:00<?, ?it/s]

100%|██████████| 695/695 [00:00<00:00, 1590.11it/s]

RMSE: 3.2044946893148825


In [74]:
non_zero_ratio = user_sparse_matrix.astype(bool).sum(axis=0) / len(user_sparse_matrix)
print("각 열의 0이 아닌 값의 비율:")
print(non_zero_ratio)
# 데이터프레임의 전체 요소 수 계산
total_elements = user_sparse_matrix.size
# 0이 아닌 값의 개수 계산
non_zero_count = user_sparse_matrix.astype(bool).sum().sum()
# 0이 아닌 값의 비율 계산
non_zero_percentage = (non_zero_count / total_elements) * 100
print("0이 아닌 값의 총 비율: {:.2f}%".format(non_zero_percentage))

각 열의 0이 아닌 값의 비율:
recipeID
1      1.000000
2      1.000000
4      1.000000
8      1.000000
9      1.000000
         ...   
865    0.006427
868    0.000756
869    0.031380
870    0.001890
872    0.002268
Length: 351, dtype: float64
0이 아닌 값의 총 비율: 1.82%


In [75]:
def get_collabor(cossim_df, title):
    return cossim_df[title].sort_values(ascending=False)[:5]

In [76]:
item_result = get_collabor(item_cossim_df,1)
item_result

recipeID
1      1.000000
25     0.839198
255    0.839198
170    0.815554
194    0.815554
Name: 1, dtype: float64

In [87]:
print(data.loc[data['recipeID'] == 1, 'category'])
print(data.loc[data['recipeID'] == 25, 'category'])
print(data.loc[data['recipeID'] == 255, 'category'])
print(data.loc[data['recipeID'] == 170, 'category'])
print(data.loc[data['recipeID'] == 194, 'category'])

1    저칼로리 다이어트
Name: category, dtype: object
25    저칼로리 다이어트
Name: category, dtype: object
255    위 건강
Name: category, dtype: object
170    변비
Name: category, dtype: object
194    골다공증
Name: category, dtype: object


In [78]:
user_cossim_df.columns

Index(['0', '1', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005',
       ...
       '981', '983', '984', '985', '986', '988', '990', '991', '992', '997'],
      dtype='object', length=2645)

In [83]:
user_result = get_collabor(user_cossim_df, '2739')
user_result

user_id
2739    1.000000
2854    0.818286
3190    0.755166
1473    0.711084
2760    0.626536
Name: 2739, dtype: float64

In [84]:
user_result.index

Index(['2739', '2854', '3190', '1473', '2760'], dtype='object', name='user_id')

In [85]:
recipe_numbers = {}

for index, ind in enumerate(user_result.index):
    recipe_numbers[f"num_recipe_{index}"] = data_re.loc[data_re['user_id'] == ind, 'recipeID'].values[0]
print(recipe_numbers)


{'num_recipe_0': 768, 'num_recipe_1': 706, 'num_recipe_2': 71, 'num_recipe_3': 768, 'num_recipe_4': 743}


In [88]:
print(data_re.loc[data_re['user_id'] == '768', 'category'])
print(data_re.loc[data_re['user_id'] == '706', 'category'])
print(data_re.loc[data_re['user_id'] == '71', 'category'])
print(data_re.loc[data_re['user_id'] == '768', 'category'])
print(data_re.loc[data_re['user_id'] == '743', 'category'])

3389    혈액순환
Name: category, dtype: object
338    노화방지
Name: category, dtype: object
1204    변비
Name: category, dtype: object
3389    혈액순환
Name: category, dtype: object
3169    혈당조절
Name: category, dtype: object
